In [ ]:
import pandas as pd

df = pd.read_csv('./data/BTCUSDT_1h.csv')
df.head()

In [ ]:
import numpy as np
# Initialize support_time column with NaN
df['support_time'] = np.nan

# Calculate support_time
for i in range(len(df)):
    low = df.loc[i, 'low_px']
    for j in range(i - 1, -1, -1):
        if df.loc[j, 'low_px'] < low < df.loc[j, 'high_px']:
            df.loc[i, 'support_time'] = df.loc[j, 'open_time']
            break
df['support'] = df.apply(
    lambda row: df[df['open_time'] == row['support_time']]['low_px'].values[0]
    if not np.isnan(row['support_time']) else row['low_px'],
    axis=1
)

In [ ]:
df['resistance_time'] = np.nan

# Calculate resistance_time
for i in range(len(df)):
    high = df.loc[i, 'high_px']
    for j in range(i - 1, -1, -1):
        if df.loc[j, 'low_px'] < high < df.loc[j, 'high_px']:
            df.loc[i, 'resistance_time'] = df.loc[j, 'open_time']
            break

df['resistance'] = df.apply(
    lambda row: df[df['open_time'] == row['resistance_time']]['high_px'].values[0]
    if not np.isnan(row['resistance_time']) else row['high_px'],
    axis=1
)

In [ ]:
import mplfinance as mpf

symbol = df['symbol'].values[0]
timestamp = df['open_time'].values[0]

df_plot = df.copy()
df_plot['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df_plot.set_index('open_time', inplace=True)
# Rename columns to match mplfinance expectations
df_plot.rename(columns={
    'open_px': 'Open',
    'high_px': 'High',
    'low_px': 'Low',
    'close_px': 'Close',
    'base_asset_volume': 'Volume'
}, inplace=True)
# Select only the required columns for plotting
df_plot = df_plot[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

apds = [mpf.make_addplot(df['support'], type='line', color='green', linestyle='dashed'),
        mpf.make_addplot(df['resistance'], type='line', color='red', linestyle='dashed')]


# Format filename
date_str = pd.to_datetime(timestamp, unit='ms').strftime('%Y-%m-%d_%H-%M-%S')
# Plot and save the candlestick chart
mpf.plot(
    df_plot,
    type='candle',
    style='charles',
    volume=True,
    title=f"{symbol} Candlestick - {date_str}",
    addplot=apds
)


In [ ]:
import pwlf

df_plot = df.copy()

x_hat = df['open_time']
model_support = pwlf.PiecewiseLinFit(df['open_time'], df['support'])
res = model_support.fit(2)
y_hat = model_support.predict(x_hat)
df['support_pred'] = y_hat

model_resistance = pwlf.PiecewiseLinFit(df['open_time'], df['resistance'])
res = model_resistance.fit(2)
x_hat = df['open_time']
y_hat = model_resistance.predict(x_hat)
df['resistance_pred'] = y_hat

df_plot['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df_plot.set_index('open_time', inplace=True)
# Rename columns to match mplfinance expectations
df_plot.rename(columns={
    'open_px': 'Open',
    'high_px': 'High',
    'low_px': 'Low',
    'close_px': 'Close',
    'base_asset_volume': 'Volume'
}, inplace=True)
# Select only the required columns for plotting
df_plot = df_plot[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

apds = [mpf.make_addplot(df['support_pred'], type='line', color='green', linestyle='dashed'),
        mpf.make_addplot(df['resistance_pred'], type='line', color='red', linestyle='dashed')]


# Format filename
date_str = pd.to_datetime(timestamp, unit='ms').strftime('%Y-%m-%d_%H-%M-%S')
# Plot and save the candlestick chart
mpf.plot(
    df_plot,
    type='candle',
    style='charles',
    volume=True,
    title=f"{symbol} Candlestick - {date_str}",
    addplot=apds
)

In [ ]:
import pwlf
import math

df_plot = df.copy()

x_hat = df['open_time']
n = len(df['open_time'])
apds = []
best_bic = float('inf')
best_model_support = None
best_segments = 0
for segment in [2, 3]:
    model_support = pwlf.PiecewiseLinFit(df['open_time'], df['support'])
    res = model_support.fit(segment)
    ssr = model_support.ssr
    k = len(model_support.beta)  # number of parameters
    # BIC = n * ln(SSE/n) + k * ln(n)
    bic = n * math.log(ssr / n) + k * math.log(n)
    if bic < best_bic:
        best_bic = bic
        best_model_support = model_support
        best_segments = segment
y_hat = best_model_support.predict(x_hat)
df[f'support_pred_{best_segments}'] = y_hat
apds.append(mpf.make_addplot(df[f'support_pred_{best_segments}'], type='line', color='green', linestyle='dashed'))

best_bic = float('inf')
best_model_resistance = None
best_segments = 0
for segment in [2, 3]:
    model_resistance = pwlf.PiecewiseLinFit(df['open_time'], df['resistance'])
    res = model_resistance.fit(segment)
    ssr = model_resistance.ssr
    k = len(model_resistance.beta)  # number of parameters
    # BIC = n * ln(SSE/n) + k * ln(n)
    bic = n * math.log(ssr / n) + k * math.log(n)
    if bic < best_bic:
        best_bic = bic
        best_model_resistance = model_resistance
        best_segments = segment

y_hat = best_model_resistance.predict(x_hat)
df[f'resistance_pred_{best_segments}'] = y_hat
apds.append(mpf.make_addplot(df[f'resistance_pred_{best_segments}'], type='line', color='red', linestyle='dashed'))

df_plot['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df_plot.set_index('open_time', inplace=True)
# Rename columns to match mplfinance expectations
df_plot.rename(columns={
    'open_px': 'Open',
    'high_px': 'High',
    'low_px': 'Low',
    'close_px': 'Close',
    'base_asset_volume': 'Volume'
}, inplace=True)
# Select only the required columns for plotting
df_plot = df_plot[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

# Format filename
date_str = pd.to_datetime(timestamp, unit='ms').strftime('%Y-%m-%d_%H-%M-%S')
# Plot and save the candlestick chart
mpf.plot(
    df_plot,
    type='candle',
    style='charles',
    volume=True,
    title=f"{symbol} Candlestick - {date_str}",
    addplot=apds
)